In [3]:
from core.llm_client import LLMClient
import os

# model_type = os.getenv("MODEL_TYPE", "openai")
model_type = os.getenv("", "gemini")
llm_client = LLMClient(model_type=model_type)

model_type

'gemini'

In [4]:
from prompts.discovery import SYSTEM_PROMPT
from tools.registry_old import ToolRegistry
from utility import load_json

registry_json = load_json(os.path.join("tools", "registry.json"))
spec_json = load_json(os.path.join("tools", "specs.json"))

In [3]:
user_prompt_1 = str({
    "id": 1,
    "type": "straightforward",
    "conversation_context": [
      {"role": "user", "content": "Show me Apple laptops under $1200 with at least 16GB RAM"}
    ],
    "current_query": "Show me Apple laptops under $1200 with at least 16GB RAM",
    "slots_till_now": {},
    "fsm_state": "COLLECTING",
    "available_tools": registry_json,
    "spec_keys": [spec.lower() for spec in spec_json["electronics"]["laptop"]] # planner knows the subcategory
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_1)
print(response)

```json
{
  "response": "Okay, I can find Apple laptops under $1200 with at least 16GB RAM.",
  "updated_slots": {
    "category": "electronics",
    "subcategory": "laptop",
    "brand": "Apple",
    "budget": 1200,
    "ram_gb": 16
  },
  "constraints": [
    {
      "key": "brand",
      "op": "==",
      "value": "Apple",
      "dtype": "enum",
      "source": "current_query"
    },
    {
      "key": "price",
      "op": "<=",
      "value": 1200,
      "dtype": "float",
      "source": "current_query"
    },
    {
      "key": "ram_gb",
      "op": ">=",
      "value": 16,
      "dtype": "int",
      "source": "current_query"
    }
  ],
  "chosen_tool": {
    "name": "filter_products",
    "params": {
      "mandatory": {
        "subcategory": "laptop",
        "category": "electronics"
      },
      "optional": {
        "brand": "Apple",
        "price_range": [
          0,
          1200
        ],
        "specifications": [
          {
            "key": "ram_gb",
       

In [4]:
1

1

In [5]:
user_prompt_2 = str({
    "id": 2,
    "type": "straightforward",
    "conversation_context": [
      {"role": "user", "content": "I want a treadmill between $300 and $700 with a 3-year warranty"}
    ],
    "current_query": "I want a treadmill between $300 and $700 with a 3-year warranty",
    "slots_till_now": {},
    "fsm_state": "COLLECTING",
    "available_tools": registry_json,
    "spec_keys": [spec.lower() for spec in spec_json["sports"]["treadmill"]] # planner knows the subcategory
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_2)
print(response)

```json
{
  "response": "Okay, I can find treadmills between $300 and $700. Unfortunately, I can't filter by warranty duration.",
  "updated_slots": {
    "category": "sports",
    "subcategory": "treadmill",
    "budget": 700
  },
  "constraints": [
    {
      "key": "price",
      "op": "BETWEEN",
      "value": [
        300.0,
        700.0
      ],
      "dtype": "float",
      "source": "current_query"
    }
  ],
  "chosen_tool": {
    "name": "filter_products",
    "params": {
      "mandatory": {
        "subcategory": "treadmill",
        "category": "sports"
      },
      "optional": {
        "price_range": [
          300,
          700
        ]
      }
    }
  },
  "execute_flag": true,
  "tool_needed": null,
  "clarification_question": null,
  "one_shot_optional_prompt": null,
  "suggested_footnotes": [
    "Top by reviews",
    "Low return rate",
    "Short review summary"
  ]
}
```


In [6]:
user_prompt_3 = str({
    "id": 3,
    "type": "missing_optional",
    "conversation_context": [
      {"role": "user", "content": "Show me laptops"}
    ],
    "current_query": "Show me laptops",
    "slots_till_now": {},
    "fsm_state": "COLLECTING",
    "available_tools": registry_json,
    "spec_keys": [spec.lower() for spec in spec_json["electronics"]["laptop"]] # planner knows the subcategory
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_3)
print(response)

```json
{
  "response": "I can show you laptops. Do you have a preferred brand or budget in mind?",
  "updated_slots": {
    "category": "electronics",
    "subcategory": "laptop"
  },
  "constraints": [],
  "chosen_tool": {
    "name": "filter_products",
    "params": {
      "mandatory": {
        "subcategory": "laptop",
        "category": "electronics"
      },
      "optional": {}
    }
  },
  "execute_flag": false,
  "tool_needed": null,
  "clarification_question": null,
  "one_shot_optional_prompt": "Do you have a preferred brand or budget?",
  "suggested_footnotes": [
    "Top by reviews",
    "Low return rate",
    "Short review summary"
  ]
}
```


In [11]:
spec_json

{'electronics': {'laptop': ['Processor',
   'RAM',
   'Storage',
   'Graphics',
   'Display',
   'Operating System',
   'Weight'],
  'phone': ['Processor',
   'RAM',
   'Storage',
   'Display',
   'Camera',
   'Battery',
   'Operating System']},
 'sports': {'treadmill': ['Motor',
   'Speed Range',
   'Incline Range',
   'Running Surface',
   'Display',
   'Incline'],
  'earphone': ['Driver Size',
   'Connectivity',
   'Battery Life',
   'Noise Cancellation',
   'Water Resistance',
   'Charging']}}

In [13]:
spec_json["electronics"]

{'laptop': ['Processor',
  'RAM',
  'Storage',
  'Graphics',
  'Display',
  'Operating System',
  'Weight'],
 'phone': ['Processor',
  'RAM',
  'Storage',
  'Display',
  'Camera',
  'Battery',
  'Operating System']}

In [5]:
user_prompt_4 = str({
    "id": 4,
    "type": "missing_optional",
    "conversation_context": [
      {"role": "user", "content": "I want headphones"}
    ],
    "current_query": "I want headphones",
    "slots_till_now": {},
    "fsm_state": "COLLECTING",
    "available_tools": registry_json,
    "spec_keys": [spec.lower() for spec in spec_json["electronics"]["earphone"]] # planner knows the subcategory from current query
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_4)
print(response)

```json
{
  "response": "I can show you headphones. Do you have a preferred brand or price range?",
  "updated_slots": {
    "category": "electronics",
    "subcategory": "headphones"
  },
  "constraints": [],
  "chosen_tool": {
    "name": "filter_products",
    "params": {
      "mandatory": {
        "subcategory": "headphones",
        "category": "electronics"
      },
      "optional": {}
    }
  },
  "execute_flag": false,
  "tool_needed": null,
  "clarification_question": null,
  "one_shot_optional_prompt": "Do you have a preferred brand or price range?",
  "suggested_footnotes": [
    "Top by reviews",
    "Low return rate",
    "Short review summary"
  ]
}
```


In [7]:
user_prompt_5 = str({
    "id": 5,
    "type": "ambiguity",
    "conversation_context": [
      {"role": "user", "content": "What are the best mobiles?"}
    ],
    "current_query": "What are the best mobiles?",
    "slots_till_now": {},
    "fsm_state": "COLLECTING",
    "available_tools": ["rank_products_by_reviews","rank_products_by_review_count","filter_products"],
    "spec_keys": [spec.lower() for spec in spec_json["electronics"]["smartphone"]] # planner knows the subcategory from current query
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_5)
print(response)

```json
{
  "response": "To clarify, are you looking for mobiles with the highest ratings or the most reviews?",
  "updated_slots": {
    "category": "electronics",
    "subcategory": "mobile"
  },
  "constraints": [],
  "chosen_tool": null,
  "execute_flag": false,
  "tool_needed": "rank_products_by_reviews",
  "clarification_question": "Do you want to rank by average rating or by review count?",
  "one_shot_optional_prompt": null,
  "suggested_footnotes": [
    "Top by reviews",
    "Low return rate",
    "Short review summary"
  ]
}
```


In [8]:
user_prompt_6 = str({
    "id": 6,
    "type": "ambiguity",
    "conversation_context": [
      {"role": "user", "content": "Which headphones are most popular?"}
    ],
    "current_query": "Which headphones are most popular?",
    "slots_till_now": {},
    "fsm_state": "COLLECTING",
    "available_tools": ["rank_products_by_review_count","product_cooccurrence_analysis"],
    "spec_keys": [spec.lower() for spec in spec_json["electronics"]["earphone"]] # planner knows the subcategory from current query
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_6)
print(response)

```json
{
  "response": "When you say 'most popular', do you mean the headphones with the highest number of reviews or something else?",
  "updated_slots": {
    "category": "electronics",
    "subcategory": "headphones"
  },
  "constraints": [],
  "chosen_tool": null,
  "execute_flag": false,
  "tool_needed": "rank_products_by_review_count",
  "clarification_question": "Do you want to rank by review count?",
  "one_shot_optional_prompt": null,
  "suggested_footnotes": [
    "Top by reviews",
    "Low return rate",
    "Short review summary"
  ]
}
```


In [9]:
user_prompt_7 = str({
    "id": 7,
    "type": "context_switch",
    "conversation_context": [
      {"role": "user", "content": "Show me Dell laptops under $1000"},
      {"role": "assistant", "content": "Do you want any specific RAM or screen size?"},
      {"role": "user", "content": "Actually, show me mobiles"}
    ],
    "current_query": "Actually, show me mobiles",
    "slots_till_now": {"subcategory":"laptop","brand":"Dell","budget":1000},
    "fsm_state": "COLLECTING",
    "available_tools": ["filter_products","list_brands"],
    "spec_keys": [spec.lower() for spec in spec_json["electronics"]["smartphone"]] # planner knows the subcategory
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_7)
print(response)

```json
{
  "response": "Got it, switching to mobiles. Do you want to filter by brand or budget?",
  "updated_slots": {
    "category": "electronics",
    "subcategory": "mobile"
  },
  "constraints": [],
  "chosen_tool": null,
  "execute_flag": false,
  "tool_needed": "filter_products",
  "clarification_question": "Do you have a preferred brand or budget for mobiles?",
  "one_shot_optional_prompt": "Do you want to specify a budget or brand?",
  "suggested_footnotes": [
    "Top by reviews",
    "Low return rate",
    "Short review summary"
  ]
}
```


In [10]:
user_prompt_8 = str({
    "id": 8,
    "type": "context_switch",
    "conversation_context": [
      {"role": "user", "content": "I'm looking for a cricket bat"},
      {"role": "assistant", "content": "What weight do you prefer?"},
      {"role": "user", "content": "Instead, show me a treadmill"}
    ],
    "current_query": "Instead, show me a treadmill",
    "slots_till_now": {"subcategory":"cricket_bat"},
    "fsm_state": "COLLECTING",
    "available_tools": ["filter_products","list_specifications"],
    "spec_keys": [spec.lower() for spec in spec_json["sports"]["treadmill"]] # planner knows the subcategory
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_8)
print(response)

```json
{
  "response": "Okay, switching to treadmills. Do you have a preferred maximum speed or budget?",
  "updated_slots": {
    "category": "sports",
    "subcategory": "treadmill"
  },
  "constraints": [],
  "chosen_tool": {
    "name": "filter_products",
    "params": {
      "mandatory": {
        "subcategory": "treadmill",
        "category": "sports"
      },
      "optional": {}
    }
  },
  "execute_flag": false,
  "tool_needed": "filter_products",
  "clarification_question": "Do you have a preferred maximum speed or budget for treadmills?",
  "one_shot_optional_prompt": "Do you want to specify a budget or max speed?",
  "suggested_footnotes": [
    "Top by reviews",
    "Low return rate",
    "Short review summary"
  ]
}
```


In [11]:
user_prompt_9 = str({
    "id": 9,
    "type": "contradiction_update",
    "conversation_context": [
      {"role": "user", "content": "Show me laptops under $2000"},
      {"role": "user", "content": "Actually under $1000"}
    ],
    "current_query": "Actually under $1000",
    "slots_till_now": {"subcategory":"laptop","budget":2000},
    "fsm_state": "COLLECTING",
    "available_tools": ["filter_products"],
    "spec_keys": [spec.lower() for spec in spec_json["electronics"]["laptop"]] # planner knows the subcategory
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_9)
print(response)

```json
{
  "response": "Okay, I’ll update the budget to $1000 for laptops. Shall I search now?",
  "updated_slots": {
    "category": "electronics",
    "subcategory": "laptop",
    "budget": 1000
  },
  "constraints": [
    {
      "key": "price",
      "op": "<=",
      "value": 1000.0,
      "dtype": "float",
      "source": "current_query"
    }
  ],
  "chosen_tool": {
    "name": "filter_products",
    "params": {
      "mandatory": {
        "subcategory": "laptop",
        "category": "electronics"
      },
      "optional": {
        "price_range": [
          0,
          1000.0
        ]
      }
    }
  },
  "execute_flag": true,
  "tool_needed": null,
  "clarification_question": null,
  "one_shot_optional_prompt": "Would you also like to add a brand or RAM size?",
  "suggested_footnotes": [
    "Top by reviews",
    "Low return rate",
    "Short review summary"
  ]
}
```


In [12]:
user_prompt_10 = str({
    "id": 10,
    "type": "contradiction_update",
    "conversation_context": [
      {"role": "user", "content": "I want a phone with 8GB RAM"},
      {"role": "user", "content": "No, make it 12GB"}
    ],
    "current_query": "No, make it 12GB",
    "slots_till_now": {"subcategory":"mobile","ram_gb":8},
    "fsm_state": "COLLECTING",
    "available_tools": ["filter_products"],
    "spec_keys": [spec.lower() for spec in spec_json["electronics"]["smartphone"]] # planner knows the subcategory
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_10)
print(response)

```json
{
  "response": "Okay, I'll update the RAM to 12GB for the mobile. Shall I search now?",
  "updated_slots": {
    "category": "electronics",
    "subcategory": "mobile",
    "ram_gb": 12
  },
  "constraints": [
    {
      "key": "ram_gb",
      "op": "==",
      "value": 12,
      "dtype": "int",
      "source": "current_query"
    }
  ],
  "chosen_tool": {
    "name": "filter_products",
    "params": {
      "mandatory": {
        "subcategory": "mobile",
        "category": "electronics"
      },
      "optional": {
        "specifications": [
          {
            "key": "ram_gb",
            "op": "==",
            "value": 12
          }
        ]
      }
    }
  },
  "execute_flag": true,
  "tool_needed": null,
  "clarification_question": null,
  "one_shot_optional_prompt": "Do you have a preferred brand?",
  "suggested_footnotes": [
    "Top by reviews",
    "Low return rate",
    "Short review summary"
  ]
}
```


In [13]:
user_prompt_11 = str({
    "id": 11,
    "type": "out_of_scope",
    "conversation_context": [
      {"role": "user", "content": "How many orders did I place last year?"}
    ],
    "current_query": "How many orders did I place last year?",
    "slots_till_now": {},
    "fsm_state": "COLLECTING",
    "available_tools": ["filter_products","rank_products_by_reviews"],
    "spec_keys": {}
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_11)
print(response)

```json
{
  "response": "We do not have a relevant tool to answer this query, please ask something else.",
  "updated_slots": {},
  "constraints": [],
  "chosen_tool": null,
  "execute_flag": false,
  "tool_needed": null,
  "clarification_question": null,
  "one_shot_optional_prompt": null,
  "suggested_footnotes": []
}
```


In [14]:
user_prompt_12 = str({
    "id": 12,
    "type": "out_of_scope",
    "conversation_context": [
      {"role": "user", "content": "Which of my past purchases were returned?"}
    ],
    "current_query": "Which of my past purchases were returned?",
    "slots_till_now": {"user_id": 555},
    "fsm_state": "COLLECTING",
    "available_tools": ["filter_products","get_product_reviews"],
    "spec_keys": {}
  })

response = await llm_client.generate(SYSTEM_PROMPT, user_prompt_12)
print(response)

```json
{
  "response": "We do not have a relevant tool to answer this query, please ask something else.",
  "updated_slots": {},
  "constraints": [],
  "chosen_tool": null,
  "execute_flag": false,
  "tool_needed": null,
  "clarification_question": null,
  "one_shot_optional_prompt": null,
  "suggested_footnotes": []
}
```


In [8]:
1

1